# scikit-learnのトレーニング♨

## 様々な分類器（PPN、SVM、DTL、LRA） 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
- [共通的データ処理](#共通的データ処理)
- [モデル・アルゴリズム](#モデル・アルゴリズム)
  - PPN: [パーセプトロン](#単純パーセプトロン線形分類器)
  - SVM: [サポートベクターマシン](#サポートベクターマシン分類器)
  - DTL: [決定木（分類木）](#決定木分析)
  - LRA: [ロジスティクス回帰](#ロジスティック回帰)
- その他
  - [各特徴量の重要度を出力](#各特徴量の重要度を出力)
  - [分類問題の性能の評価](#分類問題の性能の評価)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python#y4054d35

## [環境準備](ScikitLearnTraining0.ipynb)

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets                                 # サンプル・データセット
from sklearn.datasets import make_regression                 # 回帰データセット作成
from sklearn.datasets import make_blobs                      # 分類データセット作成
from sklearn.preprocessing import StandardScaler             # 標準化
from sklearn.model_selection import train_test_split         # データ分割

from sklearn.linear_model import LinearRegression            # 線形回帰
from sklearn.preprocessing import PolynomialFeatures         # 多項式回帰の変数変換
from sklearn.linear_model import Ridge                       # 多項式回帰のRidge回帰
from sklearn.linear_model import Lasso                       # 多項式回帰のLASSO回帰
from sklearn.linear_model import ElasticNet                  # 多項式回帰のLASSO回帰
from sklearn.linear_model import Perceptron                  # 単純パーセプトロン線形分類器
from sklearn.linear_model import LogisticRegression          # ロジスティク回帰
from sklearn.svm import SVC                                  # サポートベクターマシン（SVM）分類器
from sklearn.tree import DecisionTreeClassifier              # 決定木（分類木）
from sklearn.ensemble import RandomForestClassifier          # ランダムフォレスト（分類木）
from sklearn.ensemble import GradientBoostingClassifier      # 勾配ブースティング木（分類木）
from sklearn.decomposition import PCA                        # 主成分分析
from sklearn.cluster import KMeans                           # k-means法 クラスタ分析
from sklearn.feature_extraction.text import CountVectorizer  # 自然言語処理ベクトル化
from sklearn.feature_extraction.text import TfidfTransformer # 自然言語ベクトルのTF-IDF計算
from sklearn.decomposition import LatentDirichletAllocation  # 自然言語ベクトルからLDAトピック抽出

from sklearn import metrics                                  # モデル評価
from sklearn.metrics import mean_squared_error as mse        # 精度評価（mse）
from sklearn.metrics import confusion_matrix                 # 混同行列
from sklearn.metrics import silhouette_samples               # シルエット係数
from sklearn.model_selection import cross_val_score          # 交差検証法
from sklearn.model_selection import KFold                    # k分割交差検証法
from sklearn.model_selection import StratifiedKFold          # 層化交差検証法
from sklearn.model_selection import GridSearchCV             # グリッドサーチ

import seaborn as sns                                        # matplotlibラッパ
from mlxtend.plotting import plot_decision_regions           # 決定領域表示関数
from matplotlib import cm                                    # カラーマップ処理

from numpy import linalg as LA                               # 線形代数ライブラリ

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 共通的データ処理

### 生成

In [ ]:
iris = datasets.load_iris()
df_data = pd.DataFrame(iris.data, columns=iris.feature_names)
df_target = pd.DataFrame(iris.target, columns=['species'])
df = pd.concat([df_data, df_target], axis=1)
df

### 加工
特になし。

### 理解

#### 基本統計量

In [ ]:
df.describe()

#### 相関

##### カテゴリ分類した散布図行列を表示

In [ ]:
sns.pairplot(df, hue='species')
plt.show()

### 準備

#### 説明変数・目的変数の選択・分割

In [ ]:
np_arr=np.array(df)

# n = 100で2値分類
# n = 150で3値分類
n = 150

# PetalLengthCm, PetalWidthCm列の選択
x=np_arr[:n, 2:4] 
# x=np_arr[:n, 0:4] 

# Species列の選択
y=np.array(np_arr[:n, 4:5],dtype=np.int64) # 要素の型をint64に変換

#### 標準化
カテゴリ・データは対象外

In [ ]:
ss = StandardScaler()
ss.fit(x)
x_std = ss.transform(x)

#### 学習・テストデータの分割

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_std, y, test_size=0.3, random_state=0)

## モデル・アルゴリズム
- ココでは、以下のデータで分類するが、説明変数を増やしても良い。
- ただし、説明変数を2以上に増やすと、平面上で確認ができなくなる。

In [ ]:
plt.scatter(x_std[:50, 0], x_std[:50,1], color="red", marker="s", label="setosa")
plt.scatter(x_std[50:100, 0], x_std[50:100,1], color="blue", marker="x", label="versicolor")
plt.scatter(x_std[100:150, 0], x_std[100:150,1], color="yellow", marker="o", label="virginica")
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.show()

### 単純パーセプトロン線形分類器
- 単層パーセプトロンだから線形問題しか解けない模様。
- パーセプトロンやニューラルネットワークについては[コチラ](KasagoDL1.ipynb)を参照。

※ [イメージ](https://www.google.com/search?q=単純パーセプトロン線形分類器&tbm=isch)

#### [データ](共通的データ処理)

#### 実行

##### 学習

In [ ]:
ppn = Perceptron(eta0=0.1) # 学習率 0.1
ppn.fit(x_train, np.reshape(y_train,(-1)))

##### 推論

In [ ]:
index = 15
print('answer : %d' % y_test[index][0])
print('predict: %d' % ppn.predict([x_test[index]])[0])

#### 評価

##### データセットの正答率

In [ ]:
print('train acc: %.3f' % ppn.score(x_train, y_train))
print('test acc: %.3f' % ppn.score(x_test, y_test))

##### 学習した決定境界を可視化

In [ ]:
plot_decision_regions(x_std, y.flatten(), ppn)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

### サポートベクターマシン分類器
- ｢分離マージン最大化｣というコンセプトに基いて超平面で分類を実行。
- 非線形の分離にも対応できるという非常に大きな利点がある｡

※ [イメージ](https://www.google.com/search?q=サポートベクターマシン&tbm=isch)、[説明](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%B5%B1%E8%A8%88%E8%A7%A3%E6%9E%90#f3322edd)

#### [データ](共通的データ処理)

#### 実行

##### 学習

###### 通常

In [ ]:
svc = SVC(kernel='linear')
svc.fit(x_train, np.reshape(y_train,(-1))) 

###### スラック変数の導入（誤分類を許容）
C: 誤分類を調整する正則化パラメタ（小は誤分類に寛大、大は誤分類に厳格）

In [ ]:
svc1 = SVC(kernel='linear', C=1.0)
svc1.fit(x_train, np.reshape(y_train,(-1))) 

###### カーネル法の導入（非線形対応）
C: 誤分類への厳しさ、gamma: 決定曲線の複雑さ

In [ ]:
svc2 = SVC(kernel='rbf', gamma=0.1, C=10)
svc2.fit(x_train, np.reshape(y_train,(-1))) 

##### 推論

In [ ]:
# 通常
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % svc.predict([x_test[index]])[0])

In [ ]:
# スラック変数の導入（誤分類を許容）
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % svc1.predict([x_test[index]])[0])

In [ ]:
# カーネル法の導入（非線形対応）
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % svc2.predict([x_test[index]])[0])

#### 評価

##### データセットの正答率

In [ ]:
# 通常
print('train acc: %.3f' % svc.score(x_train, y_train))
print('test acc: %.3f' % svc.score(x_test, y_test))

In [ ]:
# スラック変数の導入（誤分類を許容）
print('train acc: %.3f' % svc1.score(x_train, y_train))
print('test acc: %.3f' % svc1.score(x_test, y_test))

In [ ]:
# カーネル法の導入（非線形対応）
print('train acc: %.3f' % svc2.score(x_train, y_train))
print('test acc: %.3f' % svc2.score(x_test, y_test))

##### 学習した決定境界を可視化

In [ ]:
# 通常
plot_decision_regions(x_std, y.flatten(), svc)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

In [ ]:
# スラック変数の導入（誤分類を許容）
plot_decision_regions(x_std, y.flatten(), svc1)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

In [ ]:
# カーネル法の導入（非線形対応）
plot_decision_regions(x_std, y.flatten(), svc2)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

##### カーネル法をXORパターンで評価
カーネル法で以下のテストデータを使うと、  
決定境界の非線形性が顕著に可視化される。

###### XORのデータの作成

In [ ]:
np.random.seed(0)
x_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(x_xor[:, 0] > 0, x_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)
# データの散布
plt.scatter(x_xor[y_xor == 1, 0], x_xor[y_xor == 1, 1], c='b', marker='x', label='1')
plt.scatter(x_xor[y_xor == -1, 0], x_xor[y_xor == -1, 1], c='r', marker='s', label='-1')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc='best') # 右上に凡例を出力
plt.show()

###### ホールド・アウト法による学習・テストデータの分割

In [ ]:
x_xor_train, x_xor_test, y_xor_train, y_xor_test = train_test_split(x_xor, y_xor, test_size=0.3, random_state=0)

###### カーネルSVMによる学習
C: 誤分類への厳しさ、gamma: 決定曲線の複雑さ

In [ ]:
svc3 = SVC(kernel='rbf', gamma=0.1, C=10)
svc3.fit(x_xor_train, y_xor_train)

###### 決定曲線のプロット

In [ ]:
plot_decision_regions(x_xor, y_xor, svc3)

### 決定木分析
決定木（分類木）による分類器で分類

※ [イメージ](https://www.google.com/search?q=決定木（分類木）&tbm=isch)、[説明](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%B5%B1%E8%A8%88%E8%A7%A3%E6%9E%90#lcaf7a28)

#### [データ](共通的データ処理)

#### 実行

##### 学習

###### 通常

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(x_train, y_train)

###### 剪定
max_depthを指定

In [ ]:
tree_md = DecisionTreeClassifier(random_state=0, max_depth=3)
tree_md.fit(x_train, y_train)

###### ランダムフォレスト
n_estimatorsを指定

In [ ]:
tree_rfc = RandomForestClassifier(random_state=0, n_estimators=10)
tree_rfc.fit(x_train, np.reshape(y_train,(-1)))

###### 勾配ブースティング木
learning_rateを指定。

In [ ]:
tree_gbc = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
tree_gbc.fit(x_train, np.reshape(y_train,(-1)))

##### 推論

###### 通常

In [ ]:
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % tree.predict([x_test[index]])[0])

###### 剪定

In [ ]:
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % tree_md.predict([x_test[index]])[0])

###### ランダムフォレスト

In [ ]:
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % tree_rfc.predict([x_test[index]])[0])

###### 勾配ブースティング木

In [ ]:
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % tree_gbc.predict([x_test[index]])[0])

#### 評価

##### データセットの正答率

###### 通常

In [ ]:
print('train acc: %.3f' % tree.score(x_train, y_train))
print('test acc: %.3f' % tree.score(x_test, y_test))

###### 剪定

In [ ]:
print('train acc: %.3f' % tree_md.score(x_train, y_train))
print('test acc: %.3f' % tree_md.score(x_test, y_test))

###### ランダムフォレスト

In [ ]:
print('train acc: %.3f' % tree_rfc.score(x_train, y_train))
print('test acc: %.3f' % tree_rfc.score(x_test, y_test))

###### 勾配ブースティング木

In [ ]:
print('train acc: %.3f' % tree_gbc.score(x_train, y_train))
print('test acc: %.3f' % tree_gbc.score(x_test, y_test))

##### 学習した決定境界を可視化

###### 通常

In [ ]:
plot_decision_regions(x_std, y.flatten(), tree)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

###### 剪定

In [ ]:
plot_decision_regions(x_std, y.flatten(), tree_md)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

###### ランダムフォレスト

In [ ]:
plot_decision_regions(x_std, y.flatten(), tree_rfc)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

###### 勾配ブースティング木

In [ ]:
plot_decision_regions(x_std, y.flatten(), tree_gbc)
plt.xlabel("PetalLengthCm")
plt.ylabel("PetalWidthCm")
plt.legend(loc="upper left")
plt.title('Iris')
plt.show()

##### 予測確率を出力可能
ランダムフォレスト、勾配ブースティング木のみの機能

###### ランダムフォレスト

In [ ]:
tree_rfc.predict_proba(x_test[11].reshape(1, -1))

###### 勾配ブースティング木

In [ ]:
tree_gbc.predict_proba(x_test[11].reshape(1, -1))

##### 各特徴量の重要度を出力_
[ロジスティクス回帰](#ロジスティック回帰)のデータを使用するなどして（[後述](#各特徴量の重要度を出力)）。

###### 配列で表示

In [ ]:
print(tree.feature_importances_)

###### 可視化（棒グラフ）
[ロジスティクス回帰](#ロジスティック回帰)の一番最後から実行

In [ ]:
# x_columns = len(df.columns)
# plt.figure(figsize=(12, 8))
# plt.barh(range(x_columns), tree.feature_importances_ , align='center')
# plt.yticks(np.arange(x_columns), df_x.columns)
# plt.show()

### ロジスティック回帰
- 判定確率を求めることが出来る線形分類器
- 最尤推定という柔軟に最適解を求められる手法を採用

※ [イメージ](https://www.google.com/search?q=ロジスティック回帰&tbm=isch)、[説明](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python#we38b247)

#### データ
ここだけ、違うデータを使用する。

##### 生成

In [ ]:
breast_cancer = datasets.load_breast_cancer()
df_data = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df_target = pd.DataFrame(breast_cancer.target, columns=['diagnosis'])
df = pd.concat([df_data, df_target], axis=1)
df.head()

##### 加工
なし

##### 理解

In [ ]:
#df.columns

In [ ]:
df_pickup = df.loc[:, ['mean concave points', 'worst radius', 'worst perimeter', 'worst concave points', 'diagnosis']]
sns.pairplot(df_pickup, hue='diagnosis')
plt.show()

##### 準備

###### 選択

In [ ]:
x = df.loc[:, ['worst radius', 'worst concave points']].values
y = df.loc[:, ['diagnosis']].values

###### 標準化
カテゴリ・データは対象外

In [ ]:
ss = StandardScaler()
ss.fit(x)
x_std = ss.transform(x)

###### ホールド・アウト法による学習・テストデータの分割

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_std, y, test_size=0.3, random_state=0)

#### 実行

##### 学習

In [ ]:
lr = LogisticRegression(C=1.0)
lr.fit(x_train, np.reshape(y_train,(-1)))

##### 推論
ココでは２値分類

###### 実行

In [ ]:
index = 10
print('answer : %d' % y_test[index][0])
print('predict: %d' % lr.predict([x_test[index]])[0])

###### 確率

In [ ]:
lr.predict_proba([x_test[index]])[0]

#### 評価

##### データセットの正答率

In [ ]:
print('train acc: %.3f' % lr.score(x_train, y_train))
print('test acc: %.3f' % lr.score(x_test, y_test))

##### 学習した決定境界を可視化

In [ ]:
plot_decision_regions(x_std, y.flatten(), lr)
plt.xlabel("perimeter_worst")
plt.ylabel("concave points_mean")
plt.legend(loc="upper left")
plt.title('cancer')
plt.show()

## その他

### 各特徴量の重要度を出力
これは、RandomForestClassifierの機能

In [ ]:
df_pickup_x = df_pickup.drop(['diagnosis'], axis=1)
df_pickup_y = df_pickup.loc[:, ['diagnosis']]
x = df_pickup_x.values
y = df_pickup_y.values
tree_rfc = RandomForestClassifier(random_state=0, n_estimators=100)
tree_rfc.fit(x, y)

#### 配列で表示

In [ ]:
df_pickup_x.columns

In [ ]:
print(tree_rfc.feature_importances_)

#### 可視化（棒グラフ）

In [ ]:
x_columns = len(df_pickup_x.columns)
plt.figure(figsize=(12, 8))
plt.barh(range(x_columns), tree_rfc.feature_importances_ , align='center')
plt.yticks(np.arange(x_columns), df_pickup_x.columns)
plt.show()

### [分類問題の性能の評価](ScikitLearnTraining5.ipynb)